In [208]:
import pandas as pd
import numpy as np
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import scipy.stats as stats
import data_cleaning
%matplotlib inline



In [209]:
df = pd.read_csv('datasets/train.csv')
df.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [210]:
snake_df(df)

In [211]:
#look at all your data that is already numeric
df.select_dtypes(np.number)

,id,pid,ms_subclass,lot_frontage,lot_area,overall_qual,overall_cond,year_built,year_remod/add,mas_vnr_area,...,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,misc_val,mo_sold,yr_sold,saleprice
0,109,533352170,60,NaN,13517,6,8,1976,2005,289.0,...,0,44,0,0,0,0,0,3,2010,130500
1,544,531379050,60,43.0,11492,7,5,1996,1997,132.0,...,0,74,0,0,0,0,0,4,2009,220000
2,153,535304180,20,68.0,7922,5,7,1953,2007,0.0,...,0,52,0,0,0,0,0,1,2010,109000
3,318,916386060,60,73.0,9802,5,5,2006,2007,0.0,...,100,0,0,0,0,0,0,4,2010,174000
4,255,906425045,50,82.0,14235,6,8,1900,1993,0.0,...,0,59,0,0,0,0,0,3,2010,138500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046,1587,921126030,20,79.0,11449,8,5,2007,2007,0.0,...,0,276,0,0,0,0,0,1,2008,298751
2047,785,905377130,30,NaN,12342,4,5,1940,1950,0.0,...,158,0,0,0,0,0,0,3,2009,82500
2048,916,909253010,50,57.0,7558,6,6,1928,1950,0.0,...,0,0,0,0,0,0,0,3,2009,177000
2049,639,535179160,20,80.0,10400,4,5,1956,1956,0.0,...,0,189,140,0,0,0,0,11,2009,144000


In [213]:
#how many null/missing values in the numeric data?
df.select_dtypes(np.number).isnull().sum()

id                   0
pid                  0
ms_subclass          0
lot_frontage       330
lot_area             0
overall_qual         0
overall_cond         0
year_built           0
year_remod/add       0
mas_vnr_area        22
bsmtfin_sf_1         1
bsmtfin_sf_2         1
bsmt_unf_sf          1
total_bsmt_sf        1
1st_flr_sf           0
2nd_flr_sf           0
low_qual_fin_sf      0
gr_liv_area          0
bsmt_full_bath       2
bsmt_half_bath       2
full_bath            0
half_bath            0
bedroom_abvgr        0
kitchen_abvgr        0
totrms_abvgrd        0
fireplaces           0
garage_yr_blt      114
garage_cars          1
garage_area          1
wood_deck_sf         0
open_porch_sf        0
enclosed_porch       0
3ssn_porch           0
screen_porch         0
pool_area            0
misc_val             0
mo_sold              0
yr_sold              0
saleprice            0
dtype: int64

In [214]:
#ferret out the garage data.
df[(df['garage_yr_blt'].isnull()) & 
   (df['garage_type'] != 'NA') ][['year_built','year_remod/add']]

,year_built,year_remod/add
28,1895,2006
53,1970,1970
65,2007,2007
79,1925,2003
101,1920,2002
...,...,...
1991,1971,1971
2010,1945,1950
2027,1960,1960
2039,1970,1970


In [215]:
df.loc[(df['garage_yr_blt'].isnull()) & 
       (df['garage_type'] != 'NA'),'garage_yr_blt'] = 1923
df[(df['garage_yr_blt'].isnull()) & (df['garage_type'] != 'NA') ]

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice


In [216]:
#no basement, no sq footage, bathrooms, etc!
df.loc[df['bsmtfin_sf_1'].isnull(),'bsmt_qual']

1327    NaN
Name: bsmt_qual, dtype: object

In [217]:
df.loc[df['bsmt_full_bath'].isnull(),'bsmt_qual']

616     NaN
1327    NaN
Name: bsmt_qual, dtype: object

In [236]:
#fill some NA's that need to be there
na_dict = {'garage_type' : 'NA',
           'lot_frontage' : 0,
           'mas_vnr_area': 0,
           'bsmtfin_sf_1': 0,
           'garage_yr_blt': 'NA',
           'bsmt_full_bath': 'NA',
           'bsmt_half_bath': 'NA',
           'bsmtfin_sf_1': 'NA',
           'bsmtfin_sf_2': 'NA',
           'bsmt_unf_sf': 'NA',
           'total_bsmt_sf': 'NA',
           'garage_cars': df[df['garage_cars'] == df['garage_cars'].value_counts().index[0]]['garage_area'].mean()
          }
df.fillna(na_dict, inplace=True)
#df[['Lot Frontage','Mas Vnr Area']].isnull().sum()

In [222]:
#now how many null numerical values?
df.select_dtypes(np.number).isnull().sum()

id                 0
pid                0
ms_subclass        0
lot_frontage       0
lot_area           0
overall_qual       0
overall_cond       0
year_built         0
year_remod/add     0
mas_vnr_area       0
1st_flr_sf         0
2nd_flr_sf         0
low_qual_fin_sf    0
gr_liv_area        0
full_bath          0
half_bath          0
bedroom_abvgr      0
kitchen_abvgr      0
totrms_abvgrd      0
fireplaces         0
garage_yr_blt      0
garage_cars        0
garage_area        0
wood_deck_sf       0
open_porch_sf      0
enclosed_porch     0
3ssn_porch         0
screen_porch       0
pool_area          0
misc_val           0
mo_sold            0
yr_sold            0
saleprice          0
dtype: int64

In [223]:
df.to_csv('datasets/train_cleaned_numerical.csv', index=False)

In [224]:
# now let's check out categorical data!
df.select_dtypes(object).isnull().sum()

ms_zoning            0
street               0
alley             1911
lot_shape            0
land_contour         0
utilities            0
lot_config           0
land_slope           0
neighborhood         0
condition_1          0
condition_2          0
bldg_type            0
house_style          0
roof_style           0
roof_matl            0
exterior_1st         0
exterior_2nd         0
mas_vnr_type        22
exter_qual           0
exter_cond           0
foundation           0
bsmt_qual           55
bsmt_cond           55
bsmt_exposure       58
bsmtfin_type_1      55
bsmtfin_sf_1         0
bsmtfin_type_2      56
bsmtfin_sf_2         0
bsmt_unf_sf          0
total_bsmt_sf        0
heating              0
heating_qc           0
central_air          0
electrical           0
bsmt_full_bath       0
bsmt_half_bath       0
kitchen_qual         0
functional           0
fireplace_qu      1000
garage_type          0
garage_finish      114
garage_qual        114
garage_cond        114
paved_drive

In [225]:
df[df['mas_vnr_type'].isnull()]['mas_vnr_area']

22      0.0
41      0.0
86      0.0
212     0.0
276     0.0
338     0.0
431     0.0
451     0.0
591     0.0
844     0.0
913     0.0
939     0.0
1025    0.0
1244    0.0
1306    0.0
1430    0.0
1434    0.0
1606    0.0
1699    0.0
1815    0.0
1820    0.0
1941    0.0
Name: mas_vnr_area, dtype: float64

In [226]:
bsmt = ['bsmt_qual',
        'bsmt_cond',
        'bsmt_exposure',
        'bsmtfin_type_1',
        'bsmtfin_sf_1',
        'bsmtfin_type_2'
       ]

df[df['bsmt_qual'].isnull()][bsmt]

,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2
12,NaN,NaN,NaN,NaN,0,NaN
93,NaN,NaN,NaN,NaN,0,NaN
114,NaN,NaN,NaN,NaN,0,NaN
146,NaN,NaN,NaN,NaN,0,NaN
183,NaN,NaN,NaN,NaN,0,NaN
240,NaN,NaN,NaN,NaN,0,NaN
249,NaN,NaN,NaN,NaN,0,NaN
256,NaN,NaN,NaN,NaN,0,NaN
390,NaN,NaN,NaN,NaN,0,NaN
437,NaN,NaN,NaN,NaN,0,NaN


In [227]:
df[df['fireplace_qu'].isnull()]['fireplaces']

0       0
2       0
3       0
4       0
6       0
       ..
2034    0
2038    0
2039    0
2044    0
2047    0
Name: fireplaces, Length: 1000, dtype: int64

In [228]:
df[df['garage_finish'].isnull()]['garage_type']


28      NA
53      NA
65      NA
79      NA
101     NA
        ..
1991    NA
2010    NA
2027    NA
2039    NA
2042    NA
Name: garage_type, Length: 114, dtype: object

In [229]:
df['alley'].value_counts()

grvl    85
pave    55
Name: alley, dtype: int64

In [230]:
df['pool_qc'].value_counts()

gd    4
fa    2
ta    2
ex    1
Name: pool_qc, dtype: int64

In [231]:
df['fence'].value_counts()

mnprv    227
gdprv     83
gdwo      80
mnww      10
Name: fence, dtype: int64

In [232]:
df['misc_feature'].value_counts()

shed    56
gar2     4
othr     3
elev     1
tenc     1
Name: misc_feature, dtype: int64

In [233]:
#fill some NA's that need to be there
na_dict = {'bsmt_qual': 'NA',
           'bsmt_cond': 'NA',
           'bsmt_exposure': 'NA',
           'bsmtfin_type_1': 'NA',
           'bsmtfin_type_2': 'NA',
           'mas_vnr_type': 'None',
           'fireplace_qu': 'NA',
           'garage_finish': 'NA',
           'garage_qual': 'NA',
           'garage_cond': 'NA',
           'garage_type': 'NA',
           'alley': 'NA',
           'pool_qc': 'NA',
           'fence': 'NA',
           'misc_feature': 'NA',
           'electrical': df['electrical'].value_counts().index[0],
           'central_air':
           
           
           
           
           
          }
df.fillna(na_dict, inplace=True)


In [196]:
# now let's re-check out categorical data!
df.select_dtypes(object).isnull().sum()

ms_zoning         0
street            0
alley             0
lot_shape         0
land_contour      0
utilities         0
lot_config        0
land_slope        0
neighborhood      0
condition_1       0
condition_2       0
bldg_type         0
house_style       0
roof_style        0
roof_matl         0
exterior_1st      0
exterior_2nd      0
mas_vnr_type      0
exter_qual        0
exter_cond        0
foundation        0
bsmt_qual         0
bsmt_cond         0
bsmt_exposure     0
bsmtfin_type_1    0
bsmtfin_type_2    0
heating           0
heating_qc        0
central_air       0
electrical        0
kitchen_qual      0
functional        0
fireplace_qu      0
garage_type       0
garage_finish     0
garage_qual       0
garage_cond       0
paved_drive       0
pool_qc           0
fence             0
misc_feature      0
sale_type         0
dtype: int64

In [91]:
#save your fresh and clean dataset
df.to_csv('datasets/train_cleaned.csv', index=False)

In [199]:
#df.to_csv('datasets/test_cleaned.csv', index=False)